# ACS Serial Stream Example - With Known Device File

`acpype` assumes a default baudrate of 115200 bps for the ACS. 

Some convenience utilities are provided for finding a COM port that an ACS is attached to `acpype.utils.find_acs_port`. This is done by looking for the ACS packet registration packets over 1 second on an incomming stream. Note that this function will iterate through the available ports and will return the first available ACS port that is found. It is not guaranteed to work with multiple ACS instruments unless the port is first made unavailable.

In [1]:
from acpype import ACSDev, ACSStream, parse_packet, calibrate_packet
from acpype.utils import find_acs_port

## Import Device File
The corresponding device file can be read in as a text file and parsed into an `ACSDev` object. This object is used to calibrate the data that is read from the ACS.
The ACSDev object contains all of the device file information, parsed out into Pythonic datatypes. It also provides linear interpolation functions for deltaT correction. To visualize and store the device file as an Xarray.Dataset, the to.xarray() method can be used.

In [2]:
dev_filepath = 'data/dev/ACS-00011_2022-10-20.DEV'
dev = ACSDev(dev_filepath)

In [3]:
devds = dev.to_xarray()
print(devds)

<xarray.Dataset> Size: 50kB
Dimensions:          (a_wavelength: 84, c_wavelength: 84, temperature_bin: 35)
Coordinates:
  * a_wavelength     (a_wavelength) float64 672B 401.8 405.3 ... 735.0 738.9
  * c_wavelength     (c_wavelength) float64 672B 400.1 403.7 ... 734.2 738.1
  * temperature_bin  (temperature_bin) float64 280B 0.7502 1.331 ... 33.52 34.45
Data variables:
    a_offset         (a_wavelength) float64 672B 0.7493 0.8166 ... 0.4245 0.2605
    a_delta_t        (a_wavelength, temperature_bin) float64 24kB -7.9e-05 .....
    c_offset         (c_wavelength) float64 672B 0.6014 0.6657 ... -0.9383
    c_delta_t        (c_wavelength, temperature_bin) float64 24kB 0.05002 ......
Attributes: (12/25)
    device_filepath:                data/dev/ACS-00011_2022-10-20.DEV
    sensor_type:                    ACS Meter
    serial_number_hexdec:           5300000B
    serial_number:                  ACS-00011
    device_file_structure_version:  3
    tcal:                           22.3
    .

## Acquire, parse, and calibrate 8 packets.

In [4]:
calibrated_data = []
port = find_acs_port()  # Find the port that has an ACS attached to it.
if port is not None: # If a port is found and it is streaming ACS registration bytes...
    print(f'ACS available on port {port}.\n')
    i = 0
    with ACSStream(port) as acss: # Open a serial object.
        while True:  # Continuously read the stream until a full packet is found.
            acss.read_stream()
            acs_packet = acss.find_packet()
            if acs_packet.packet_data is not None:
                parsed_packet = parse_packet(acs_packet)
                calibrated_packet = calibrate_packet(parsed_packet, dev)
                calibrated_data.append(calibrated_packet)

                # Code to store parsed and calibrated packet data would go here and you would remove the break statement below.
  
                i += 1
                if i == 8:
                    break# Break after finding the first 8 full length packets, otherwise this example would continue forever.
                    
    [print(f'{cal_data}\n') for cal_data in calibrated_data] # Print the calibrated packet data.       
else:
    print('No ACS found on any port.')

ACS available on port COM5.

CalibratedPacket(daq_time=datetime.datetime(2025, 4, 22, 18, 17, 33, 133467, tzinfo=datetime.timezone.utc), serial_number='ACS-00011', flag_syntax=1, elapsed_time=2466049, flag_elapsed_time=1, internal_temperature=25.877998785441605, flag_internal_temperature=1, external_temperature=25.89493843207144, a_wavelength=(401.8, 405.3, 408.7, 412.1, 415.5, 419.0, 423.3, 427.6, 431.3, 435.4, 439.1, 442.8, 447.4, 451.8, 455.9, 459.9, 464.1, 468.4, 472.6, 477.3, 482.0, 486.4, 490.6, 494.8, 498.8, 502.8, 506.9, 511.8, 516.5, 521.0, 524.9, 529.2, 532.9, 537.2, 541.3, 545.2, 549.7, 553.8, 558.2, 562.5, 566.2, 570.3, 573.7, 577.3, 581.0, 584.6, 586.9, 590.7, 595.0, 599.6, 603.8, 608.0, 612.3, 616.9, 621.2, 625.2, 629.4, 633.4, 637.3, 641.6, 646.0, 650.9, 655.2, 660.9, 665.0, 668.5, 672.9, 677.1, 681.1, 685.5, 689.2, 692.6, 696.3, 700.4, 704.4, 707.8, 711.6, 715.3, 719.5, 723.5, 727.3, 731.1, 735.0, 738.9), c_wavelength=(400.1, 403.7, 407.1, 410.7, 413.7, 417.6, 421.5, 42